# Recreating Experiment 1 in Crowdsourcing Interface Feature Design with Bayesian Optimization

- Paper: https://jsonj.co.uk/resources/science/papers/chi2019-bayesian-optimization.pdf
- Experimental Data: https://www.repository.cam.ac.uk/handle/1810/292232

The code below is a **very simplified** attempt to recreate this paper's Experiment 1. See the discussion below the code for aspects of the paper that were simplified.





In [ ]:
import numpy as np
import scipy.stats
from matplotlib import pyplot as plt
import GPy; import GPyOpt

In [ ]:
def f_u(x):
    print(x)
    while True:
        res = input('Completion Time: ')
        res = float(res)
        return res


def run_bo(max_iter):
    bounds = [{'name': 'x1', 'type': 'continuous', 'domain': (10, 200)},
              {'name': 'x2', 'type': 'continuous', 'domain': (100, 1000)},
              {'name': 'x3', 'type': 'continuous', 'domain': (0.5, 2.5)},
              {'name': 'x4', 'type': 'continuous', 'domain': (100, 1000)},
              {'name': 'x5', 'type': 'continuous', 'domain': (0.2,1)}]
    myBopt = GPyOpt.methods.BayesianOptimization(
        f=f_u, domain=bounds,
        acquisition_type='EI',
        exact_feval=False,
        eps=1e-6,
        normalize_Y=False,
        initial_design_numdata=2)
    myBopt.run_optimization(max_iter=max_iter - 2)

    return myBopt

if __name__ == "__main__":
    n_iter = 8
    # run BO
    bo = run_bo(n_iter)
    bo_xs, bo_ys = bo.get_evaluations()

    plt.plot(-bo_ys, 'k-', label='BO')
    plt.xlabel('iterations')
    plt.ylabel('completion time')
    plt.show()

## RESCALING OBSERVATIONS 
> We rescale the observation values to have zero mean and unit variance. To do this in the absence
of prior data, we require a coarse approximation of the distribution of typical observation values.
In this paper, the observation values are task completion times. Completion time is
approximately the product of the number of inspections and
time per inspection. Such products approach a log-normal
distribution. Based on initial pilot testing and for consistency
across the experiments and case study, we assume a mean
completion time of approximately 30 s. To normalize for unit
variance we expect typical task times might vary between
15 s (half) and 60 s (double) which corresponds very approximately with a log-normal standard deviation of 0.7.

<p>Time constraints didn't allow me to learn how to appropriately implement this.

## BATCHING
> Within each batch and for each
user, however, the standard process of Bayesian optimization
also incorporates the individual user’s prior performance.
The hyperparameters are held constant during a batch and
then updated based on all data up to and including the most
recent batch. In the first batch with no prior data, the hyperparameters were all set to a nominal value (unity).

<p>I've asked the authors to clarify as what constitutes an observation.

## KERNEL AND ACQUISITION FUNCTION
>In this study we employ the automatic relevance determination (ARD) kernel
The literature provides many choices for acquisition function. We use a
standard approach based on expected improvement (EI).

<p>

>From Brochu et al., 2010a
Note that the conventional approach to setting
RBF parameters (c,α) as well as the kernel parameters (θ,σ) is to maximize the log-likelihood of the
model within a specific optimization run. This method works
well for many application of GPs, but unfortunately for our
application, it is known to work poorly when the number
of training data is small (exactly the situation we are in).
The sparsity of data in the space can lead to the likelihood
function becoming very flat on some dimensions, or even
monotonically increasing to infinity. This can lead to low quality models
or ill-conditioned covariance matrices.

<p>"ARD Kernel" could be a misnomer as it appears to be an attribute of multiple kernels in GPy. As a simplification, I used the default kernel.

## OPTIMIZING THE ACQUISITION FUNCTION
> The approach involves evaluating a candidate list of sample
points that provide representative coverage of the design space.
Appropriate bounds for each parameter are chosen
and this sets the limits of the hypercube. The candidate list
is then constructed by sampling from the parameter hypercube using a Sobol sequence as described by 26. We use
1000 candidates sampled in this way.
    
<p>Time constraints didn't allow me to learn how to appropriately implement this.